In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import joblib

In [9]:
# -------------------------------
# Load datasets (base for SARIMA, features for ML)
# -------------------------------
df_base = pd.read_csv("../data/processed/cleaned_data.csv")       # has Material_Name
df_feat = pd.read_csv("../data/processed/feature_data.csv")       # one-hot features
df_base["Date"] = pd.to_datetime(df_base["Date"])
df_feat["Date"] = pd.to_datetime(df_feat["Date"])

print("Base (clean) shape:", df_base.shape)
print("Feature (ML) shape:", df_feat.shape)

Base (clean) shape: (2193, 9)
Feature (ML) shape: (2103, 19)


In [10]:
# -------------------------------
# Time-based split (align both)
# -------------------------------
split_date = pd.Timestamp("2024-07-01")

base_train = df_base[df_base["Date"] < split_date].copy()
base_test  = df_base[df_base["Date"] >= split_date].copy()

feat_train = df_feat[df_feat["Date"] < split_date].copy()
feat_test  = df_feat[df_feat["Date"] >= split_date].copy()

target = "Quantity_Consumed"
feature_cols = [c for c in df_feat.columns if c not in ["Date", target]]

print("Train/Test shapes -> base:", base_train.shape, base_test.shape, "| feat:", feat_train.shape, feat_test.shape)


Train/Test shapes -> base: (1641, 9) (552, 9) | feat: (1551, 19) (552, 19)


In [11]:
# -------------------------------
# 1) SARIMA per material (using base data with Material_Name)
# -------------------------------
sarima_preds_list = []
for mat in base_train["Material_Name"].unique():
    mt_train = base_train[base_train["Material_Name"] == mat].sort_values("Date").set_index("Date")
    mt_test  = base_test [base_test ["Material_Name"] == mat].sort_values("Date").set_index("Date")

    # Defensive: need non-empty series
    if len(mt_train) == 0 or len(mt_test) == 0:
        print(f"Skip SARIMA for {mat} due to insufficient data.")
        continue

    try:
        # Lightweight seasonal model to keep it fast on i3/8GB
        model = SARIMAX(mt_train[target], order=(1,1,1), seasonal_order=(1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
        res = model.fit(disp=False)
        preds = res.forecast(steps=len(mt_test))
        tmp = mt_test[[target]].copy()
        tmp["Material_Name"] = mat
        tmp = tmp.reset_index()
        tmp.rename(columns={target: "y_true"}, inplace=True)
        tmp["SARIMA_Pred"] = preds.values
        sarima_preds_list.append(tmp[["Date", "Material_Name", "y_true", "SARIMA_Pred"]])
        print(f"SARIMA done for {mat} (n_test={len(mt_test)})")
    except Exception as e:
        print(f"⚠️ SARIMA failed for {mat}: {e}")

sarima_df = pd.concat(sarima_preds_list, ignore_index=True) if sarima_preds_list else pd.DataFrame()
print("SARIMA predictions shape:", sarima_df.shape)


C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


SARIMA done for TMT_Steel (n_test=184)


C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


SARIMA done for Cement (n_test=184)


C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


SARIMA done for Sand (n_test=184)
SARIMA predictions shape: (552, 4)


In [12]:
# -------------------------------
# 2) ML models (RF + XGBoost) on feature data
# -------------------------------
X_tr, y_tr = feat_train[feature_cols], feat_train[target]
X_te, y_te = feat_test [feature_cols], feat_test [target]

rf = RandomForestRegressor(n_estimators=160, random_state=42, n_jobs=-1)
rf.fit(X_tr, y_tr)
rf_preds = rf.predict(X_te)

xg = xgb.XGBRegressor(
    n_estimators=240, learning_rate=0.05, max_depth=5,
    subsample=0.8, colsample_bytree=0.9, random_state=42, n_jobs=-1
)
xg.fit(X_tr, y_tr)
xg_preds = xg.predict(X_te)

# Simple hybrid of RF + XGB
hyb_preds = (rf_preds + xg_preds) / 2


In [14]:

# -------------------------------
# 3) Evaluation helpers
# -------------------------------
def eval_metrics(y_true, y_pred, label):
    mse = mean_squared_error(y_true, y_pred)   # no 'squared' kwarg
    rmse = np.sqrt(mse)                        # turn MSE -> RMSE
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f"{label:12s} | RMSE: {rmse:8.3f} | MAPE: {mape*100:6.2f}%")
    return rmse, mape

print("\n=== ML Evaluation on test period ===")
rf_scores  = eval_metrics(y_te, rf_preds, "RandomForest")
xgb_scores = eval_metrics(y_te, xg_preds, "XGBoost")
hyb_scores = eval_metrics(y_te, hyb_preds, "Hybrid(RF+XGB)")


=== ML Evaluation on test period ===
RandomForest | RMSE:    5.686 | MAPE:   4.02%
XGBoost      | RMSE:    5.638 | MAPE:   4.01%
Hybrid(RF+XGB) | RMSE:    5.589 | MAPE:   3.99%


In [15]:
# SARIMA eval (merge by Date+Material_Name to align truth)
if not sarima_df.empty:
    # Build ground truth frame keyed by Date+Material_Name from base_test
    truth = base_test[["Date", "Material_Name", target]].copy()
    truth.rename(columns={target: "y_true"}, inplace=True)
    sarima_eval = sarima_df.merge(truth, on=["Date", "Material_Name", "y_true"], how="inner")
    s_rmse, s_mape = eval_metrics(sarima_eval["y_true"], sarima_eval["SARIMA_Pred"], "SARIMA")
else:
    print("No SARIMA predictions available for evaluation.")
    s_rmse, s_mape = (np.nan, np.nan)

SARIMA       | RMSE:   26.219 | MAPE:  21.47%


In [16]:
# -------------------------------
# 4) Save models
# -------------------------------
os.makedirs("../models", exist_ok=True)
joblib.dump(rf, "../models/rf_model.pkl")
joblib.dump(xg, "../models/xgb_model.pkl")
print("✅ Models saved to ../models/")


✅ Models saved to ../models/


In [17]:
# -------------------------------
# 6) Save test-set predictions for evaluation notebook (Step 6)
# -------------------------------

# Build tidy predictions DataFrame for ML models
ml_preds_df = pd.DataFrame({
    "Date": feat_test["Date"].reset_index(drop=True),
    "y_true": y_te.reset_index(drop=True),
    "RF_Pred": rf_preds,
    "XGB_Pred": xg_preds,
    "HYB_Pred": (rf_preds + xg_preds) / 2
})

os.makedirs("../data/processed", exist_ok=True)
ml_preds_df.to_csv("../data/processed/test_predictions_ml.csv", index=False)

# Also persist SARIMA predictions if available
if not sarima_df.empty:
    sarima_df.to_csv("../data/processed/test_predictions_sarima.csv", index=False)

print("✅ Saved test predictions to data/processed/ for Step 6.")


✅ Saved test predictions to data/processed/ for Step 6.
